## IMPORTS

In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import zipfile
import os
import matplotlib as ml
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, RandomTreesEmbedding
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score, precision_recall_curve, roc_curve, classification_report

## GETING DATA

In [89]:
def load_train():
    return pd.read_csv('datasets/train.csv')

def load_test():
    return pd.read_csv('datasets/test.csv')

train = load_train()
test = load_test()

## SETS

In [90]:
# Dividindo o conjunto de dados em treinamento e teste

treino, validacao = train_test_split(train, test_size=0.3)


X_treino = treino.drop('Survived', axis=1)
y_treino = treino['Survived']

X_valid = validacao.drop(columns=['Survived'], axis=1)
y_valid = validacao['Survived']


## TRANSFORMING DATA

In [91]:
labels = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
numerical_labels = ['Age', 'Fare']
categorical_labels = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

In [92]:
num_pipe = Pipeline(steps=[
    ('imp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

standard_scaler = Pipeline(steps=[('scaler', StandardScaler())])

In [93]:
preprocess = ColumnTransformer(transformers=[
    ('fill_median', fill_median, numerical_labels),
    ('standard_scaler', standard_scaler, numerical_labels)
])

## CROSS-VAL-SCORE

In [94]:
def cross_validate_models(model):

   pipeline = Pipeline(steps=[
      ('preprocess', preprocess),
      ('classifier', model)
   ])
   
   pipeline.fit(X_treino, y_treino)
   y_predict = pipeline.predict(X_valid)
   print(precision_score(y_valid, y_predict))

In [95]:
cross_validate_models(RandomForestClassifier())

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').